Se va a seguir el tutorial encontrado en [https://ai.google.dev/edge/mediapipe/solutions/vision/hand_landmarker/python](https://ai.google.dev/edge/mediapipe/solutions/vision/hand_landmarker/python)

Instalamos la librería necesaria

In [ ]:
!python -m pip install mediapipe
!pip install opencv-python

Importamos librerías y modelo de reconocimiento de mano

In [30]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

model_path = '/content/drive/MyDrive/PFG/Hand Recording Demo/hand_landmarker.task'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Create a hand landmarker instance with the video mode:
options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.VIDEO)
with HandLandmarker.create_from_options(options) as landmarker:
    # El landmarker está inicializado. Aquí puedes procesar las imágenes.
    # Por ejemplo, para procesar una imagen:
    # image = cv2.imread('your_image.jpg')  # Cargar imagen (asegúrate de usar OpenCV u otra biblioteca para cargar imágenes)
    # results = landmarker.detect(image)
    # ... (Aquí iría el procesamiento adicional)
    pass

Procesar el vídeo

In [35]:
import cv2
from google.colab.patches import cv2_imshow

# Inicializar MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Cargar el video
video_path = '/content/drive/MyDrive/PFG/Hand Recording Demo/hand_rec_demo_video.mp4'
cap = cv2.VideoCapture(video_path)

# Lista para almacenar las coordenadas
data = []

# Obtener propiedades del video original
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Definir el codec y crear un VideoWriter para guardar el video procesado
output_path = "/content/drive/MyDrive/PFG/Hand Recording Demo/video_procesado_demo.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec para MP4
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Configurar MediaPipe Hands
with mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5) as hands:
    frame_id = 0  # Contador de frames

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_id += 1  # Aumentar contador de frames
        print(f"Procesando frame {frame_id}")

        # Convertir BGR a RGB para MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Procesar el frame con MediaPipe Hands
        results = hands.process(frame_rgb)

        # Extraer coordenadas si se detectan manos
        if results.multi_hand_landmarks:
            for hand_idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
                for i, landmark in enumerate(hand_landmarks.landmark):
                    data.append({
                        "frame": frame_id,
                        "hand_id": hand_idx,
                        "landmark_id": i,
                        "x": landmark.x,
                        "y": landmark.y,
                        "z": landmark.z
                    })

        # Dibujar landmarks si se detectan manos
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Guardar el frame en el nuevo video
        out.write(frame)

# Liberar recursos
cap.release()
out.release()

# Guardar en CSV las coordenadas
df = pd.DataFrame(data)
csv_path = "/content/drive/MyDrive/PFG/Hand Recording Demo/landmarks_demo.csv"
df.to_csv(csv_path, index=False)

print(f"Datos guardados en: {csv_path}")

print(f"Video procesado guardado en: {output_path}")

Procesando frame 1
Procesando frame 2
Procesando frame 3
Procesando frame 4
Procesando frame 5
Procesando frame 6
Procesando frame 7
Procesando frame 8
Procesando frame 9
Procesando frame 10
Procesando frame 11
Procesando frame 12
Procesando frame 13
Procesando frame 14
Procesando frame 15
Procesando frame 16
Procesando frame 17
Procesando frame 18
Procesando frame 19
Procesando frame 20
Procesando frame 21
Procesando frame 22
Procesando frame 23
Procesando frame 24
Procesando frame 25
Procesando frame 26
Procesando frame 27
Procesando frame 28
Procesando frame 29
Procesando frame 30
Procesando frame 31
Procesando frame 32
Procesando frame 33
Procesando frame 34
Procesando frame 35


KeyboardInterrupt: 